#Multiple Linear Regression and Polinomial Regression for house price prediction



In [0]:
import numpy as np
import math as math
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from google.colab import auth
from google.colab import drive
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn import linear_model

In [2]:
drive.mount('/content/gdrive') #mount the googledrive
#!ls "/content/gdrive/My Drive/DA_Coursework/3. Data" #list the content

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##1 Load Dataset

In [0]:
#s_boroughs = pd.read_csv("/content/gdrive/My Drive/DA_Coursework/3. Data/londonBoroughs.csv")
#s_boroughs = pd.read_csv("/content/gdrive/My Drive/DA_Coursework/3. Data/londonBoroughsWO_Others.csv")
#s_boroughs = pd.read_csv("/content/gdrive/My Drive/DA_Coursework/3. Data/londonWoOtherWoOutliers.csv")
s_boroughs = pd.read_csv("/content/gdrive/My Drive/DA_Coursework/3. Data/londonBoroughs_realprice.csv")

#s_boroughs

##2 Select filters

In [0]:
# CHOOSE NOMINAL OR REAL PRICE FILE
#nom_real = "londonBoroughs.csv"
#nom_real = "londonBoroughs_realprice.csv"

# CHOOSE A DISTRICT
# district = 'CITY OF WESTMINSTER'
#district = 'LAMBETH'
# district = 'CROYDON'

# SETTING PROPERTY TYPE TO "OTHERS"
property_type = 'O'

# SETTING LOWER ADN UPPER PRICE BOUNDS FOR OUTLIERS
lower_price = 20000
upper_price = 30000000

#data_df = pd.read_csv(Path + nom_real)
#data_df = data_df[data_df['district']==district]
s_boroughs = s_boroughs[s_boroughs['propertyType']!=property_type]
s_boroughs = s_boroughs[(s_boroughs['price']>=lower_price) & (s_boroughs['price']<=upper_price)]
#s_boroughs['dateOfTransfer'] = pd.to_datetime(data_df['dateOfTransfer']).map(dt.date.toordinal)

# CHOOOSE A FILTER DATE
# filter_date = '2010-01-01'
# filter_date = '2000-01-01'
# data_df = data_df[data_df['dateOfTransfer']<filter_date] # filtering for data on or after filter_date
# print('date range =', data_df['dateOfTransfer'][:1], data_df['dateOfTransfer'][-1:])

##3 Add dummy values and convert date

In [0]:
#Convert Date to numerical value for the three datasets:

s_boroughs['dateOfTransfer'] = pd.to_datetime(s_boroughs['dateOfTransfer'])
s_boroughs['dateOfTransfer'] = s_boroughs['dateOfTransfer'].map(dt.datetime.toordinal)

In [0]:
#Create the dummy values for categorical values (Property Type, Old/New, Duration, PPD)
s_boroughs = pd.concat([s_boroughs,pd.get_dummies(s_boroughs['propertyType'], prefix='prop_type')],axis=1)
s_boroughs = pd.concat([s_boroughs,pd.get_dummies(s_boroughs['oldNew'], prefix='oldNew')],axis=1)
s_boroughs = pd.concat([s_boroughs,pd.get_dummies(s_boroughs['duration'], prefix='duration')],axis=1)
#s_boroughs = pd.concat([s_boroughs,pd.get_dummies(s_boroughs['ppdCategoryType'], prefix='ppdCategoryType')],axis=1)

In [0]:
#Divide the dataset in three boroughs:
df_lambeth = s_boroughs.loc[s_boroughs['district']=='LAMBETH'] 
df_croydon = s_boroughs.loc[s_boroughs['district']=='CROYDON']
df_westminster = s_boroughs.loc[s_boroughs['district']=='CITY OF WESTMINSTER']

In [8]:
#Drop the features that will not be considered:
df_lambeth.drop(columns=['Unnamed: 0','townCity','district','county','propertyType','oldNew','duration'],inplace=True)
df_croydon.drop(columns=['Unnamed: 0','townCity','district','county','propertyType','oldNew','duration'],inplace=True)
df_westminster.drop(columns=['Unnamed: 0','townCity','district','county','propertyType','oldNew','duration'],inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
#Reset the index for the Dataframe
df_westminster.reset_index(inplace=True)
df_westminster.drop(df_westminster.columns[[0]], axis=1, inplace=True)
df_westminster.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,uniqueId,price,dateOfTransfer,postCode,PAON,SAON,street,locality,ppdCategoryType,recordStatus,prop_type_D,prop_type_F,prop_type_S,prop_type_T,oldNew_N,oldNew_Y,duration_F,duration_L,duration_U
0,{8D28EE68-8F4F-4C7D-8A33-8E6335277383},180000.0,728613,NW8 9AA,7B,NaN,ABBEY ROAD,LONDON,A,A,0,1,0,0,1,0,0,1,0
1,{958F75D1-1D97-4221-BFB4-CE0EC069B2DF},950000.0,728295,NW8 0PS,57,NaN,MARLBOROUGH PLACE,LONDON,A,A,0,0,1,0,1,0,1,0,0
2,{A5349972-B8E0-47A9-A63C-BA2794F874D2},115000.0,728568,SW1P 4AL,"GLADSTONE COURT, 97",FLAT 31,REGENCY STREET,LONDON,A,A,0,1,0,0,1,0,0,1,0
3,{47C4CF14-2F62-425B-A16A-AF5C71801DA3},250000.0,728359,W1K 6LP,"SHEPHERDS HOUSE, 20",FLAT 6,LEES PLACE,LONDON,A,A,0,1,0,0,0,1,0,1,0
4,{41B2796E-F607-4D00-89F5-AF5E4A36BCF6},185000.0,728363,W9 1EL,6,FLAT D,WARRINGTON CRESCENT,LONDON,A,A,0,1,0,0,1,0,0,1,0
5,{CCEAE9B9-75CC-42BA-B3A3-D72C2D33A664},200000.0,728520,W1U 5LA,TREBOROUGH HOUSE,FLAT 9,NOTTINGHAM PLACE,LONDON,A,A,0,1,0,0,1,0,0,1,0
6,{01C520B3-0DEF-4534-B6E0-CCA88EDCD26D},140000.0,728432,W1J 5AX,48,FLAT H,BERKELEY SQUARE,LONDON,A,A,0,1,0,0,1,0,0,1,0
7,{A11069A1-82FA-4B2A-9094-873B1AA487D2},693000.0,728473,NW1 5HN,HARLEY HOUSE,FLAT 72,MARYLEBONE ROAD,LONDON,A,A,0,1,0,0,0,1,0,1,0
8,{A6508C3A-BC42-431A-8C5D-D03426CD5EB9},66000.0,728481,W2 2SD,2,SECOND FLOOR FLAT,BATHURST STREET,LONDON,A,A,0,1,0,0,1,0,0,1,0
9,{0A961BE3-0969-42F8-B058-CCACD9F794F5},560000.0,728360,SW1W 0RB,25,NaN,VICTORIA SQUARE,LONDON,A,A,0,0,0,1,1,0,1,0,0


In [0]:
df_lambeth = df_lambeth.groupby('dateOfTransfer').mean().reset_index()
df_croydon = df_croydon.groupby('dateOfTransfer').mean().reset_index()
df_westminster = df_westminster.groupby('dateOfTransfer').mean().reset_index()


##4 Train test split dataset

In [0]:
#Split the training set and test set

Xw = df_westminster[['dateOfTransfer', 'prop_type_D', 'prop_type_F',
       'prop_type_S', 'prop_type_T', 'oldNew_N', 'oldNew_Y', 'duration_F',
       'duration_L', 'duration_U']]
yw = df_westminster.price

Xw_train, Xw_test, yw_train, yw_test = train_test_split(Xw, yw, test_size=0.2,random_state = 1)

Xc = df_croydon[['dateOfTransfer', 'prop_type_D', 'prop_type_F',
       'prop_type_S', 'prop_type_T', 'oldNew_N', 'oldNew_Y', 'duration_F',
       'duration_L', 'duration_U']]
yc = df_croydon.price

Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.2,random_state = 1)

Xl = df_lambeth[['dateOfTransfer', 'prop_type_D', 'prop_type_F',
       'prop_type_S', 'prop_type_T', 'oldNew_N', 'oldNew_Y', 'duration_F',
       'duration_L', 'duration_U']]
yl = df_lambeth.price

Xl_train, Xl_test, yl_train, yl_test = train_test_split(Xl, yl, test_size=0.2,random_state = 1)

#display(X_train.shape)
#display(X_test.shape)
#display(y_train.shape)
#display(y_test.shape)

##5 MLR model

In [12]:
#Building the model for Westminster

model1 = LinearRegression(normalize=True)
display(model1)
model1.fit(Xw_train, yw_train)

# Plot the data and the model prediction
yw_predict = model1.predict(Xw_test)

#X_test.columns
yw_predict = pd.DataFrame(yw_predict)
yw_predict.head(10)

#Building the model for Lambeth
model2 = LinearRegression(normalize=True)
model2.fit(Xl_train, yl_train)

# Plot the data and the model prediction
yl_predict = model2.predict(Xl_test)

#X_test.columns
yl_predict = pd.DataFrame(yl_predict)
yl_predict.head(10)

#Building the model for Croydon

model3 = LinearRegression(normalize=True)
model3.fit(Xc_train, yc_train)

# Plot the data and the model prediction
yc_predict = model3.predict(Xc_test)

#X_test.columns
yc_predict = pd.DataFrame(yc_predict)
yc_predict.head(10)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

,0
0,157696.0
1,92160.0
2,226816.0
3,205824.0
4,152576.0
5,112640.0
6,203776.0
7,120832.0
8,144896.0
9,117248.0


##6 RMSE calculation

In [13]:
#MSE for each linear regression
#Westminster
MSEw = math.sqrt(mean_squared_error(yw_test,yw_predict))
print('RMSE Westminster:',MSEw)
#Croydon
MSEc = math.sqrt(mean_squared_error(yc_test,yc_predict))
print('RMSE Croydon:',MSEc)
#Lambeth
MSEl = math.sqrt(mean_squared_error(yl_test,yl_predict))
print('RMSE Lambeth:',MSEl)

RMSE Westminster: 310878.6801567848
RMSE Croydon: 35230.31378006007
RMSE Lambeth: 64715.53229958661


In [14]:
#Compare actual price vs result of the regression
comparison = pd.concat([yw_test,yw_predict],1)
comparison.head(2)

,price,0
0,NaN,589824.0
1,NaN,729088.0


##7 Polinomial Regression model

In [15]:
#POLYNOMIAL REGRESSION: Building the model: Westminster


#Transform X_train and X_test into polynomials order 2
polyW = PolynomialFeatures(degree=2, interaction_only=True)
Xw_train_Poly = polyW.fit_transform(Xw_train)
Xw_test_Poly = polyW.fit_transform(Xw_test)


#Build the model
polyW_model = LinearRegression()
polyW_model.fit(Xw_train_Poly, yw_train)

#Predict Y Values
yw_PolyPredict = polyW_model.predict(Xw_test_Poly)


#X_test.columns
yw_PolyPredict = pd.DataFrame(yw_PolyPredict)
yw_PolyPredict.head(5)




,0
0,587879.836601
1,738464.586601
2,322088.836601
3,473016.836601
4,374295.836601


In [16]:
#POLYNOMIAL REGRESSION: Building the model: Lambeth


#Transform X_train and X_test into polynomials order 2
polyL = PolynomialFeatures(degree=2, interaction_only=True)
Xl_train_Poly = polyL.fit_transform(Xl_train)
Xl_test_Poly = polyL.fit_transform(Xl_test)


#Build the model
polyL_model = LinearRegression()
polyL_model.fit(Xl_train_Poly, yl_train)

#Predict Y Values
yl_PolyPredict = polyL_model.predict(Xl_test_Poly)


#X_test.columns
yl_PolyPredict = pd.DataFrame(yl_PolyPredict)
yl_PolyPredict.head(5)

,0
0,191596.514971
1,74425.139971
2,368424.639971
3,312456.889971
4,284464.264971


In [17]:
#POLYNOMIAL REGRESSION: Building the model: Croydon


#Transform X_train and X_test into polynomials order 2
polyC = PolynomialFeatures(degree=2, interaction_only=True)
Xc_train_Poly = polyC.fit_transform(Xc_train)
Xc_test_Poly = polyC.fit_transform(Xc_test)


#Build the model
polyC_model = LinearRegression()
polyC_model.fit(Xc_train_Poly, yc_train)

#Predict Y Values
yc_PolyPredict = polyC_model.predict(Xc_test_Poly)


#X_test.columns
yc_PolyPredict = pd.DataFrame(yc_PolyPredict)
yc_PolyPredict.head(5)

,0
0,157284.814983
1,90872.814983
2,214922.814983
3,203170.314983
4,152017.814983


In [18]:
#MSE for each polynomial regression
#Westminster
MSEw_poly = math.sqrt(mean_squared_error(yw_test,yw_PolyPredict))
print('RMSE poly Westminster:',MSEw_poly)

#MSE for each linear regression
#Croydon
MSEc_poly = math.sqrt(mean_squared_error(yc_test,yc_PolyPredict))
print('RMSE poly Croydon:',MSEc_poly)

#MSE for each linear regression
#Lambeth
MSEl_poly = math.sqrt(mean_squared_error(yl_test,yl_PolyPredict))
print('RMSE poly Lambeth:',MSEl_poly)



RMSE poly Westminster: 309778.7183271143
RMSE poly Croydon: 35185.63407411494
RMSE poly Lambeth: 63548.399740508015


##8 RMSE difference

In [19]:
#Print the differences in MSE of Polynomial vs Linear Regression

print('RMSE Westminster:',MSEw_poly-MSEw)
print('RMSE Croydon:',MSEc_poly-MSEc)
print('RMSE Lambeth:',MSEl_poly-MSEl)


RMSE Westminster: -1099.961829670472
RMSE Croydon: -44.67970594512735
RMSE Lambeth: -1167.1325590785927
